# Loading the dataset

In [ ]:
from datasets import load_dataset
dataset =load_dataset("tweet_eval","emotion")

# Tokenization

In [2]:
from transformers import AutoTokenizer
model_ckpt ="distilbert-base-uncased"
tokenizer =AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_ckpt)

In [3]:
def tokenize(batch):
    return tokenizer(batch["text"],padding=True,truncation=True)
tokenized_dataset=dataset.map(tokenize,batched=True,batch_size=None)

Loading cached processed dataset at C:\Users\samya\.cache\huggingface\datasets\tweet_eval\emotion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-4e41ffc7235e5843.arrow


Map:   0%|          | 0/1421 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\samya\.cache\huggingface\datasets\tweet_eval\emotion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-66bea20aaaf7f0e0.arrow


In [4]:
from transformers import TFAutoModelForSequenceClassification
num_labels =4
model=TFAutoModelForSequenceClassification.from_pretrained(model_ckpt,num_labels=num_labels)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [5]:
batch_size=64
from transformers import DataCollatorWithPadding
data_collator =DataCollatorWithPadding(tokenizer=tokenizer,return_tensors="tf")

In [13]:
tokenized_dataset.set_format(type="torch",columns=["input_ids","text","attention_mask","label"])

# Creating Tensorflow datasets

In [16]:
tf_train_dataset =tokenized_dataset["train"].to_tf_dataset(
    columns=["input_ids","attention_mask"], 
    label_cols=["label"], 
    shuffle=True, 
    batch_size=batch_size,
    collate_fn=data_collator
)
tf_valid_dataset=tokenized_dataset["validation"].to_tf_dataset(
    columns=["input_ids","attention_mask"], 
    label_cols=["label"], 
    shuffle=False, 
    batch_size=batch_size,
    collate_fn=data_collator
)

In [17]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import losses

# Compiling the model

In [18]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=5e-5),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=metrics.SparseCategoricalAccuracy()
)

# Training the model

In [23]:
model.fit(tf_train_dataset,validation_data=tf_valid_dataset,epochs=5)

Epoch 1/5
51/51 [==============================] - 439s 8s/step - loss: 0.9772 - sparse_categorical_accuracy: 0.5969 - val_loss: 0.6862 - val_sparse_categorical_accuracy: 0.7326
Epoch 2/5
51/51 [==============================] - 433s 9s/step - loss: 0.5150 - sparse_categorical_accuracy: 0.8262 - val_loss: 0.5969 - val_sparse_categorical_accuracy: 0.7968
Epoch 3/5
51/51 [==============================] - 436s 9s/step - loss: 0.2960 - sparse_categorical_accuracy: 0.9119 - val_loss: 0.6783 - val_sparse_categorical_accuracy: 0.7834
Epoch 4/5
51/51 [==============================] - 483s 9s/step - loss: 0.1729 - sparse_categorical_accuracy: 0.9493 - val_loss: 0.7322 - val_sparse_categorical_accuracy: 0.7914
Epoch 5/5
51/51 [==============================] - 510s 10s/step - loss: 0.0946 - sparse_categorical_accuracy: 0.9748 - val_loss: 0.8186 - val_sparse_categorical_accuracy: 0.7914


### Thus, the validation accuracy of our model is 79%

## This is a task related to NLP where I have used a dataset containing tweets and emotions. This is a text classification model in which I have used HuggingFace library to train the bert model. Firstly, I have preprocessed the dataset using AutoTokenizer function which converts unstructured text data into numerical array basd on mapping. Then, I have created train dataset for training our model and validation dataset for checking the accuracy of our model. After preprocessing, I have imported all the important libraries and have compiled the model using Adam optimizer, Sparse Categorical crossentropy for the loss function and sparse categorical accuracy for the metric. The final step of the model is training on the train dataset and checking the accuracy on the validation dataset. Our 